In [1]:
import pandas as pd
from krk_ml_utils.preprocessing import convert_for_text_to_text_regression
train_df = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
train_formatted_df = convert_for_text_to_text_regression(train_df, target_column="SalePrice", precision=10, drop_column_list=["Id"])



In [2]:
# Split the dataset into train/val 
train_size = int(0.8 * len(train_formatted_df))
train_set = train_formatted_df[:train_size]
val_set = train_formatted_df[train_size:]

# Save the train and validation sets to CSV files
train_set.to_csv("house-prices-advanced-regression-techniques/train_formatted.csv", index=False)
val_set.to_csv("house-prices-advanced-regression-techniques/val_formatted.csv", index=False)

In [3]:
print(train_set.shape)
print(val_set.shape)

(1168, 2)
(292, 2)


In [4]:
train_set.head()

,input,output
0,1stFlrSF: 856\n2ndFlrSF: 854\n3SsnPorch: 0\nAl...,<+><2><0><8><5><0><0><0><0><0><0><E-4>
1,1stFlrSF: 1262\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><8><1><5><0><0><0><0><0><0><E-4>
2,1stFlrSF: 920\n2ndFlrSF: 866\n3SsnPorch: 0\nAl...,<+><2><2><3><5><0><0><0><0><0><0><E-4>
3,1stFlrSF: 961\n2ndFlrSF: 756\n3SsnPorch: 0\nAl...,<+><1><4><0><0><0><0><0><0><0><0><E-4>
4,1stFlrSF: 1145\n2ndFlrSF: 1053\n3SsnPorch: 0\n...,<+><2><5><0><0><0><0><0><0><0><0><E-4>


In [5]:
val_set.head()

,input,output
1168,1stFlrSF: 1236\n2ndFlrSF: 872\n3SsnPorch: 0\nA...,<+><2><3><5><0><0><0><0><0><0><0><E-4>
1169,1stFlrSF: 1831\n2ndFlrSF: 1796\n3SsnPorch: 0\n...,<+><6><2><5><0><0><0><0><0><0><0><E-4>
1170,1stFlrSF: 1118\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><7><1><0><0><0><0><0><0><0><E-4>
1171,1stFlrSF: 1261\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><6><3><0><0><0><0><0><0><0><E-4>
1172,1stFlrSF: 625\n2ndFlrSF: 625\n3SsnPorch: 0\nAl...,<+><1><7><1><9><0><0><0><0><0><0><E-4>


In [6]:
train_set.head()

,input,output
0,1stFlrSF: 856\n2ndFlrSF: 854\n3SsnPorch: 0\nAl...,<+><2><0><8><5><0><0><0><0><0><0><E-4>
1,1stFlrSF: 1262\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><8><1><5><0><0><0><0><0><0><E-4>
2,1stFlrSF: 920\n2ndFlrSF: 866\n3SsnPorch: 0\nAl...,<+><2><2><3><5><0><0><0><0><0><0><E-4>
3,1stFlrSF: 961\n2ndFlrSF: 756\n3SsnPorch: 0\nAl...,<+><1><4><0><0><0><0><0><0><0><0><E-4>
4,1stFlrSF: 1145\n2ndFlrSF: 1053\n3SsnPorch: 0\n...,<+><2><5><0><0><0><0><0><0><0><0><E-4>


In [7]:
val_set.head()

,input,output
1168,1stFlrSF: 1236\n2ndFlrSF: 872\n3SsnPorch: 0\nA...,<+><2><3><5><0><0><0><0><0><0><0><E-4>
1169,1stFlrSF: 1831\n2ndFlrSF: 1796\n3SsnPorch: 0\n...,<+><6><2><5><0><0><0><0><0><0><0><E-4>
1170,1stFlrSF: 1118\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><7><1><0><0><0><0><0><0><0><E-4>
1171,1stFlrSF: 1261\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><6><3><0><0><0><0><0><0><0><E-4>
1172,1stFlrSF: 625\n2ndFlrSF: 625\n3SsnPorch: 0\nAl...,<+><1><7><1><9><0><0><0><0><0><0><E-4>


In [ ]:
# Check if validation set is divisible by 8
if len(val_set) % 8 != 0:
    print("Warning: Validation set length is not divisible by 8. This may cause issues with batching in some models.")
    # Remove rows until it is divisible by 8
    val_set = val_set[:len(val_set) - (len(val_set) % 8)]
    print(f"New validation set length: {len(val_set)}")

In [8]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("./rlm_tokenizer.json")

# Add new column for input/output tokens
train_set["input_tokens"] =  train_set["input"].apply(lambda x: tokenizer.encode(x).ids)
train_set["output_tokens"] = train_set["output"].apply(lambda x: tokenizer.encode(x).ids)

val_set["input_tokens"] =  val_set["input"].apply(lambda x: tokenizer.encode(x).ids)
val_set["output_tokens"] = val_set["output"].apply(lambda x: tokenizer.encode(x).ids)



/var/folders/qn/_bpj4mkj69d4_20wgymvw24c0000gn/T/ipykernel_76991/1893898104.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set["input_tokens"] =  train_set["input"].apply(lambda x: tokenizer.encode(x).ids)
/var/folders/qn/_bpj4mkj69d4_20wgymvw24c0000gn/T/ipykernel_76991/1893898104.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set["output_tokens"] = train_set["output"].apply(lambda x: tokenizer.encode(x).ids)
/var/folders/qn/_bpj4mkj69d4_20wgymvw24c0000gn/T/ipykernel_76991/1893898104.p

In [9]:
train_set.head()

,input,output,input_tokens,output_tokens
0,1stFlrSF: 856\n2ndFlrSF: 854\n3SsnPorch: 0\nAl...,<+><2><0><8><5><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 290, 4419...","[0, 50265, 50269, 50267, 50275, 50272, 50267, ..."
1,1stFlrSF: 1262\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><8><1><5><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 316, 5379...","[0, 50265, 50268, 50275, 50268, 50272, 50267, ..."
2,1stFlrSF: 920\n2ndFlrSF: 866\n3SsnPorch: 0\nAl...,<+><2><2><3><5><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 37543, 50...","[0, 50265, 50269, 50269, 50270, 50272, 50267, ..."
3,1stFlrSF: 961\n2ndFlrSF: 756\n3SsnPorch: 0\nAl...,<+><1><4><0><0><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 361, 5606...","[0, 50265, 50268, 50271, 50267, 50267, 50267, ..."
4,1stFlrSF: 1145\n2ndFlrSF: 1053\n3SsnPorch: 0\n...,<+><2><5><0><0><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 365, 1898...","[0, 50265, 50269, 50272, 50267, 50267, 50267, ..."


In [10]:
val_set.head()

,input,output,input_tokens,output_tokens
1168,1stFlrSF: 1236\n2ndFlrSF: 872\n3SsnPorch: 0\nA...,<+><2><3><5><0><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 316, 3367...","[0, 50265, 50269, 50270, 50272, 50267, 50267, ..."
1169,1stFlrSF: 1831\n2ndFlrSF: 1796\n3SsnPorch: 0\n...,<+><6><2><5><0><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 504, 2983...","[0, 50265, 50273, 50269, 50272, 50267, 50267, ..."
1170,1stFlrSF: 1118\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><7><1><0><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 11571, 39...","[0, 50265, 50268, 50274, 50268, 50267, 50267, ..."
1171,1stFlrSF: 1261\n2ndFlrSF: 0\n3SsnPorch: 0\nAll...,<+><1><6><3><0><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 316, 5606...","[0, 50265, 50268, 50273, 50270, 50267, 50267, ..."
1172,1stFlrSF: 625\n2ndFlrSF: 625\n3SsnPorch: 0\nAl...,<+><1><7><1><9><0><0><0><0><0><0><E-4>,"[0, 134, 620, 16197, 338, 23646, 35, 37329, 50...","[0, 50265, 50268, 50274, 50268, 50276, 50267, ..."


In [11]:

for i in range(len(train_set)):
    train_str = tokenizer.decode(train_set.iloc[i]["input_tokens"])
    print(train_str)
    break

1stFlrSF: 856
2ndFlrSF: 854
3SsnPorch: 0
Alley: .nan
BedroomAbvGr: 3
BldgType: 1Fam
BsmtCond: TA
BsmtExposure: 'No'
BsmtFinSF1: 706
BsmtFinSF2: 0
BsmtFinType1: GLQ
BsmtFinType2: Unf
BsmtFullBath: 1
BsmtHalfBath: 0
BsmtQual: Gd
BsmtUnfSF: 150
CentralAir: Y
Condition1: Norm
Condition2: Norm
Electrical: SBrkr
EnclosedPorch: 0
ExterCond: TA
ExterQual: Gd
Exterior1st: VinylSd
Exterior2nd: VinylSd
Fence: .nan
FireplaceQu: .nan
Fireplaces: 0
Foundation: PConc
FullBath: 2
Functional: Typ
GarageArea: 548
GarageCars: 2
GarageCond: TA
GarageFinish: RFn
GarageQual: TA
GarageType: Attchd
GarageYrBlt: 2003.0
GrLivArea: 1710
HalfBath: 1
Heating: GasA
HeatingQC: Ex
HouseStyle: 2Story
KitchenAbvGr: 1
KitchenQual: Gd
LandContour: Lvl
LandSlope: Gtl
LotArea: 8450
LotConfig: Inside
LotFrontage: 65.0
LotShape: Reg
LowQualFinSF: 0
MSSubClass: 60
MSZoning: RL
MasVnrArea: 196.0
MasVnrType: BrkFace
MiscFeature: .nan
MiscVal: 0
MoSold: 2
Neighborhood: CollgCr
OpenPorchSF: 61
OverallCond: 5
OverallQual: 7
PavedD

In [12]:

for i in range(len(train_set)):
    tokens = train_set.iloc[i]["output_tokens"]
    train_str = tokenizer.decode(tokens)
    print(f"Output tokens: {tokens}")
    print(f"Output string: {train_str}")
    break

Output tokens: [0, 50265, 50269, 50267, 50275, 50272, 50267, 50267, 50267, 50267, 50267, 50267, 50373, 2]
Output string: <+><2><0><8><5><0><0><0><0><0><0><E-4>


In [13]:
from krk_ml_utils import datasets


datasets.create_single_npz_from_dataframe(
    file_path="./house-prices-advanced-regression-techniques/train_rlm.npz",
    dataframe=train_set,
    feature_columns=["input_tokens"],
    label_columns=["output_tokens"],
    features_key="x",
    targets_key="y",
    feature_dtype="int32",
    target_dtype="int32",
    compress=True,
)


datasets.create_single_npz_from_dataframe(
    file_path="./house-prices-advanced-regression-techniques/test_rlm.npz",
    dataframe=val_set,
    feature_columns=["input_tokens"],
    label_columns=["output_tokens"],
    features_key="x",
    targets_key="y",
    feature_dtype="int32",
    target_dtype="int32",
    compress=True,
)

Creating .npz file at: './house-prices-advanced-regression-techniques/train_rlm.npz'
  - File saved successfully.
  - Features array 'x' shape: (1168,), dtype: object
  - Targets array 'y' shape: (1168,), dtype: object
Creating .npz file at: './house-prices-advanced-regression-techniques/test_rlm.npz'
  - File saved successfully.
  - Features array 'x' shape: (292,), dtype: object
  - Targets array 'y' shape: (292,), dtype: object


In [14]:
# Get the max token count for input and output in training and val set 
max_input_tokens = train_set["input_tokens"].apply(len).max()
max_output_tokens = train_set["output_tokens"].apply(len).max()
val_max_input_tokens = val_set["input_tokens"].apply(len).max()
val_max_output_tokens = val_set["output_tokens"].apply(len).max()
print(f"Max input tokens in train set: {max_input_tokens}")
print(f"Max output tokens in train set: {max_output_tokens}")
print(f"Max combined tokens in train set: {max_input_tokens + max_output_tokens}")
print(f"Max input tokens in val set: {val_max_input_tokens}")
print(f"Max output tokens in val set: {val_max_output_tokens}")
print(f"Max combined tokens in val set: {val_max_input_tokens + val_max_output_tokens}")

Max input tokens in train set: 577
Max output tokens in train set: 14
Max combined tokens in train set: 591
Max input tokens in val set: 576
Max output tokens in val set: 14
Max combined tokens in val set: 590
